<a href="https://colab.research.google.com/github/WenheLI/-/blob/main/a0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Git and Drive Integration

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import time

### Mount google drive

In [2]:
# Mount google drive to use a persistent directory structure
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Clone the assignment repository

In [ ]:
# Clone repository in google drive
%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://YOUR_TOKEN@github.com/ML-HW-SYS/a0.git #paste your github token here

### Add `src` directory to the Python import path

In [10]:
# import functions from the src directory in repo
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/ece5545/a0-WenheLI/src')

### Import functions from `src/train_utils.py`

In [11]:
import train_utils
from model import *

### Make changes to `src` files and push to repo

In [12]:
# You will likely need to set your git user identification settings
!git config --global user.email "wl692@cornell.edu"  # update with your email
!git config --global user.name "Wenhe LI"   # update with your name 

In [13]:
# To run the same tests that are run on Github Classroom
%cd /content/gdrive/MyDrive/ece5545/a0-WenheLI/
!python3 -m pytest

/content/gdrive/MyDrive/ece5545/a0-WenheLI
============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/gdrive/MyDrive/ece5545/a0-WenheLI, inifile:
plugins: typeguard-2.7.1
collected 1 item                                                               

tests/test_model.py .                                                    [100%]

=========================== 1 passed in 2.32 seconds ===========================


In [ ]:
%cd /content/gdrive/MyDrive/ece5545/a0/src
# Navigate to a0/src/model.py and fix the bug (denoted by TODO)
# in the file then commit the changes below
!git commit -am "fixed bug in model.py"
!git push

## Training Configuration

In [23]:
batch_size = 128
epochs = 5
lr = 1.0
gamma = 0.7
no_cuda = False
seed = 42
log_interval = 50
save_model = False
dry_run = False

# Derived parameters
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

## Data Loaders and Optimizer Setup

In [24]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('/content/gdrive/MyDrive/ece5545/data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('/content/gdrive/MyDrive/ece5545/data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

## Training Loop

In [26]:
import importlib
importlib.reload(train_utils)
# we use importlib to make sure that we are loading
# the latest version of train_utils after any changes
# that you may have done

# for epoch in range(1, epochs + 1):
#     train_utils.train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
#     train_utils.test(model, device, test_loader)
#     scheduler.step()

# if save_model:
#     torch.save(model.state_dict(), "mnist_cnn.pt")

<module 'train_utils' from '/content/gdrive/MyDrive/ece5545/a0-WenheLI/src/train_utils.py'>

In [42]:
# Diagnostic information about the used GPU device
!nvidia-smi

Mon Jan 24 18:52:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    65W / 149W |    876MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [28]:
import time
import statistics 


In [27]:
# GPU Benchmark
training_time_gpu = []
for epoch in range(1, epochs + 1):
    start = time.time()
    train_utils.train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
    end = time.time()
    training_time_gpu.append(end - start)
    train_utils.test(model, device, test_loader)
    scheduler.step()
   


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.206454
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.226110
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.099422
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.242742
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.202170
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.089456
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.019015
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.100273
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.075008
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.082217

Epoch 1 time = 15.21s on device cuda

Test set: Average loss: 0.0431, Accuracy: 9863/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.070831
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.090437
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.051464
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.012464
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.037985
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.121104
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.031131
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.014423
Train Epoch: 

In [36]:
mean_time = statistics.mean(training_time_gpu)
std_time = statistics.stdev(training_time_gpu)
print(f"Mean Training Time is {mean_time:.2f}\n std Training Time is {std_time:.2f}")

Mean Training Time is 15.16
 std Training Time is 0.13


In [40]:
# Init CPU model
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': batch_size}
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
device = 'cpu'
model = Net().to('cpu')
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [41]:
# CPU Benchmark
training_time_cpu = []
for epoch in range(1, epochs + 1):
    start = time.time()
    train_utils.train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
    end = time.time()
    training_time_cpu.append(end - start)
    train_utils.test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324942
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.382082
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.244864
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.207426
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.166932
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.162418
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.140420
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.089345
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.186967
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.099100

Epoch 1 time = 130.31s on device cpu

Test set: Average loss: 0.0525, Accuracy: 9832/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.062131
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.106191
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.092774
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.097923
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.060503
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.061120
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.135880
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.051813
Train Epoch: 

In [44]:
mean_time = statistics.mean(training_time_cpu)
std_time = statistics.stdev(training_time_cpu)
print(f"Mean Training Time is {mean_time:.2f}\n std Training Time is {std_time:.2f}")

Mean Training Time is 133.82
 std Training Time is 2.12


## Batch Size Experiment

In [ ]:
batch_sizes = [1, 4, 8, 16, 32, 64, 128, 256, 512]
epochs = 5
lr = 1.0
gamma = 0.7
no_cuda = True
seed = 42
log_interval = 50
save_model = False
dry_run = False
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

# Derived parameters
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)

for batch_size in batch_sizes:
  train_kwargs = {'batch_size': batch_size}
  test_kwargs = {'batch_size': batch_size}
  if use_cuda:
      cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
      train_kwargs.update(cuda_kwargs)
      test_kwargs.update(cuda_kwargs)
  train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
  test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
  training_time_batch = []
  for epoch in range(1, epochs + 1):
      start = time.time()
      train_utils.train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
      end = time.time()
      training_time_batch.append(end - start)
      train_utils.test(model, device, test_loader)
      scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.066904
Train Epoch: 1 [50/60000 (0%)]	Loss: 2.265026
Train Epoch: 1 [100/60000 (0%)]	Loss: 2.287133
Train Epoch: 1 [150/60000 (0%)]	Loss: 2.407438
Train Epoch: 1 [200/60000 (0%)]	Loss: 2.330679
Train Epoch: 1 [250/60000 (0%)]	Loss: 2.201932
Train Epoch: 1 [300/60000 (0%)]	Loss: 2.351377
Train Epoch: 1 [350/60000 (1%)]	Loss: 2.306466
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.400608
Train Epoch: 1 [450/60000 (1%)]	Loss: 2.419361
Train Epoch: 1 [500/60000 (1%)]	Loss: 2.375021
Train Epoch: 1 [550/60000 (1%)]	Loss: 2.315106
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.322089
Train Epoch: 1 [650/60000 (1%)]	Loss: 2.363594
Train Epoch: 1 [700/60000 (1%)]	Loss: 2.335497
Train Epoch: 1 [750/60000 (1%)]	Loss: 2.343467
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.329245
Train Epoch: 1 [850/60000 (1%)]	Loss: 2.302702
Train Epoch: 1 [900/60000 (2%)]	Loss: 2.355317
Train Epoch: 1 [950/60000 (2%)]	Loss: 2.331308
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.229002
Train Epoch: 1 